In [1]:
import json
import pandas as pd
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode


spark = SparkSession.builder.appName("ProcessingDataset").getOrCreate()
spark.conf.set('spark.sql.caseSensitive', True)


## Preprocessing for the Video_Games.jsonl - User Reviews Data

### Drop unnecessary column (images) in Video_Games

In [2]:
reviews_df = spark.read.json("Video_Games.jsonl")

reviews_df.printSchema()

reviews_df.show(5)

root
 |-- asin: string (nullable = true)
 |-- helpful_vote: long (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- attachment_type: string (nullable = true)
 |    |    |-- large_image_url: string (nullable = true)
 |    |    |-- medium_image_url: string (nullable = true)
 |    |    |-- small_image_url: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- verified_purchase: boolean (nullable = true)

+----------+------------+------+-----------+------+--------------------+-------------+--------------------+--------------------+-----------------+
|      asin|helpful_vote|images|parent_asin|rating|                text|    timestamp|               title|             user_id|verified_purchase|
+----------+------------+-

In [3]:
reviews_df = reviews_df.drop('images')

In [4]:
reviews_df.printSchema()
reviews_df.show(5)


root
 |-- asin: string (nullable = true)
 |-- helpful_vote: long (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- verified_purchase: boolean (nullable = true)

+----------+------------+-----------+------+--------------------+-------------+--------------------+--------------------+-----------------+
|      asin|helpful_vote|parent_asin|rating|                text|    timestamp|               title|             user_id|verified_purchase|
+----------+------------+-----------+------+--------------------+-------------+--------------------+--------------------+-----------------+
|B07DJWBYKP|           0| B07DK1H3H5|   4.0|I’m playing on ps...|1608186804795|It’s pretty sexua...|AGCI7FAH4GL5FI65H...|             true|
|B00ZS80PC2|           1| B07SRWRH5D|   5.0|Nostalgic fun.  A...|158705111

### Cleaning and modifying data

In [5]:
from pyspark.sql.functions import from_unixtime, col

Datetime Column

In [6]:
#Create a new datetime column from timestamp (Unix time)
reviews_df = reviews_df.withColumn("datetime", from_unixtime((col("timestamp") / 1000).cast("long")))
reviews_df.printSchema()
reviews_df.show(1)


root
 |-- asin: string (nullable = true)
 |-- helpful_vote: long (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- verified_purchase: boolean (nullable = true)
 |-- datetime: string (nullable = true)

+----------+------------+-----------+------+--------------------+-------------+--------------------+--------------------+-----------------+-------------------+
|      asin|helpful_vote|parent_asin|rating|                text|    timestamp|               title|             user_id|verified_purchase|           datetime|
+----------+------------+-----------+------+--------------------+-------------+--------------------+--------------------+-----------------+-------------------+
|B07DJWBYKP|           0| B07DK1H3H5|   4.0|I’m playing on ps...|1608186804795|It’s pretty sexua...|AGCI7FAH4GL5FI

review_id : id for each review

In [7]:
from pyspark.sql.functions import monotonically_increasing_id

reviews_df = reviews_df.withColumn("review_id", monotonically_increasing_id())

text_length: length of the text field

In [8]:
from pyspark.sql.functions import length

In [9]:
# Độ dài nội dung phản hồi
reviews_df = reviews_df.withColumn("text_length", length(col("text")))

rating_label: Categorize the rating score of user's review into 3 classes: negative, neutral and positive

In [10]:
from pyspark.sql.functions import when

In [11]:
reviews_df = reviews_df.withColumn(
    "rating_label",
    when(col("rating") <= 2, "negative")
    .when(col("rating") == 3, "neutral")
    .otherwise("positive")
)

### Save the cleaned version into parquet

In [12]:
reviews_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful_vote: long (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- verified_purchase: boolean (nullable = true)
 |-- datetime: string (nullable = true)
 |-- review_id: long (nullable = false)
 |-- text_length: integer (nullable = true)
 |-- rating_label: string (nullable = false)



In [13]:
#Drop the original timestamp column
reviews_df = reviews_df.drop('timestamp')

In [14]:
reviews_df = reviews_df.select("review_id", "title",\
                               "rating", "rating_label", "user_id",\
                                "asin", "parent_asin",\
                                "text", "text_length",\
                                "helpful_vote", "verified_purchase",\
                                "datetime")

In [15]:
reviews_df.show(1)

+---------+--------------------+------+------------+--------------------+----------+-----------+--------------------+-----------+------------+-----------------+-------------------+
|review_id|               title|rating|rating_label|             user_id|      asin|parent_asin|                text|text_length|helpful_vote|verified_purchase|           datetime|
+---------+--------------------+------+------------+--------------------+----------+-----------+--------------------+-----------+------------+-----------------+-------------------+
|        0|It’s pretty sexua...|   4.0|    positive|AGCI7FAH4GL5FI65H...|B07DJWBYKP| B07DK1H3H5|I’m playing on ps...|        226|           0|             true|2020-12-17 13:33:24|
+---------+--------------------+------+------------+--------------------+----------+-----------+--------------------+-----------+------------+-----------------+-------------------+
only showing top 1 row



In [16]:
reviews_df.write.mode("overwrite").parquet("Dataset/user_reviews.parquet")

## Preprocessing for the meta_Video_Games.jsonl - Product metadata

### Drop unnecessary columns (images and videos) in meta_Video_Games

In [17]:
metadata_df = spark.read.json("meta_Video_Games.jsonl")
metadata_df.printSchema()


root
 |-- author: struct (nullable = true)
 |    |-- about: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- avatar: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- average_rating: double (nullable = true)
 |-- bought_together: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- details: struct (nullable = true)
 |    |-- AC Adapter Current: string (nullable = true)
 |    |-- Access Location: string (nullable = true)
 |    |-- Accessory Connection Type: string (nullable = true)
 |    |-- Action: string (nullable = true)
 |    |-- Active Ingredients: string (nullable = true)
 |    |-- Actors: string (nullable = true)
 |    |-- Actuator Type: string (nullable = true)
 |    |-- Additional product features: string (nullable = true)
 |    |-- Adjustable Length: string (nullabl

In [18]:
metadata_df = metadata_df.drop('images', 'videos', 'details', "description", "features", "author", "subtitle", "bought_together")

In [19]:
metadata_df.printSchema()


root
 |-- average_rating: double (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- main_category: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- price: string (nullable = true)
 |-- rating_number: long (nullable = true)
 |-- store: string (nullable = true)
 |-- title: string (nullable = true)



In [21]:
metadata_df = metadata_df.select("parent_asin", "title", "average_rating", "rating_number", "main_category", "categories", "price", "store")


In [22]:
metadata_df.write.mode("overwrite").parquet("Dataset/metadata.parquet")